### Imports

In [85]:
from __future__ import print_function, division, absolute_import
import os
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.colors import SymLogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shutil import rmtree
from matplotlib.ticker import FormatStrFormatter
from math import radians, degrees, pi, ceil, sin, asin, log, log10, floor
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from scipy.spatial.distance import pdist, squareform
from scipy import stats
from pyuvwsim import (evaluate_baseline_uvw_ha_dec, 
                      convert_enu_to_ecef)
from utilities.generators import (inner_arms,
                                  inner_arms_clusters)   
from utilities.plotting import save_fig
from utilities.analysis import generate_psf_3
import oskar
import seaborn
seaborn.set(style='ticks')
seaborn.set_style('ticks', {'xtick.major.size': 5, 'ytick.major.size': 5})
seaborn.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.5})
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Settings

In [3]:
# Settings
results_dir = 'temp_results'
station_d = 45.0
lon = radians(116.63128900)
lat = radians(-26.69702400)
alt = 0.0
dec = lat
obs_length = 0.0  # Hours, assumed to be symmetric about ha = 0.0
dump_time = 10  # Minutes
num_times = 1 if obs_length * 60 < dump_time \
    else ((obs_length) * 60) // dump_time  # 1 sample every 10 min
freq_hz = 100.0e6
oversampling = 2.4
res_n = 10
# res_bmin = None
# res_bmax = None
res_bmin = 50
res_bmax = 11.0e3
r_min = 500.0
r_max = 5000.0
wavelength = 299792458.0 / freq_hz

if os.path.isdir(results_dir):
    rmtree(results_dir)

### Generate layouts

In [4]:
layouts = dict()

b, num_arms, n = 0.5, 6, 12
layouts['spiral'] = inner_arms(b, num_arms, n, r_min, r_max)

b, num_arms, clusters_per_arm, stations_per_cluster = 0.5, 3, 4, 6
cluster_d, station_r = 200, station_d / 2
layouts['clusters'] = inner_arms_clusters(b, num_arms, clusters_per_arm, 
                                 stations_per_cluster, cluster_d, station_r,
                                 r_min, r_max)

# b, num_arms, clusters_per_arm, stations_per_cluster = 0.5, 3, 8, 3
# cluster_d, station_r = 200, station_d / 2
# layouts[2] = inner_arms_clusters(b, num_arms, clusters_per_arm, 
#                                  stations_per_cluster, cluster_d, station_r,
#                                  r_min, r_max)

for i, name in enumerate(layouts):
    print(i, name, layouts[name]['x'].shape[0])

0 clusters 72
1 spiral 72


### Network graph

In [11]:
def plot_network(x, y, tree):
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(111)
    for sx, sy in zip(x, y):
        c = plt.Circle((sx, sy), station_d/2, fill=True, color='k', 
                       alpha=1.0)
        ax.add_artist(c)     
    for i in range(y.shape[0]):
        for j in range(x.shape[0]):
            if tree[i, j] > 0:
                ax.plot([x[i], x[j]], [y[i], y[j]], 'r-', alpha=0.5, lw=1.0)
    ax.set_xlim(-r_max*1.05, r_max*1.05)
    ax.set_ylim(-r_max*1.05, r_max*1.05)
    ax.set_xlabel('east (m)')
    ax.set_ylabel('north (m)')
    save_fig(fig, 'network_%s.png' % name, [results_dir, 'network'])
    # plt.show()
    plt.close(fig)

# Network for the layout
for i, name in enumerate(layouts):
    layout = layouts[name]
    x, y = layout['x'], layout['y']
    coords = np.transpose(np.vstack((x, y)))
    tree = minimum_spanning_tree(squareform(pdist(coords))).toarray()
    layout['cable_length'] = np.sum(tree)
    print(i, layout['cable_length'] / 1.0e3)
    plot_network(x, y, tree)


0 20.2607111548
1 56.3336975823


### Generate uvw coordinates

In [5]:
def uvw_coords(layout, obs_length, num_times, lon, dec):
    """Update the layout by adding uvw coordinates."""
    x, y = layout['x'], layout['y']
    z = layout['z'] if 'z' in layout else np.zeros_like(x)        
    coord_type = layout['coord_type'] if 'coord_type' in layout else 'enu'
    if coord_type == 'enu':
        x, y, z = convert_enu_to_ecef(x, y, z, lon, lat, alt)       
    uu, vv, ww = np.array([]), np.array([]), np.array([])
    ha_off = ((obs_length / 2) / 24) * (2 * pi)
    for i, ha in enumerate(np.linspace(-ha_off, ha_off, num_times)):
        uu_, vv_, ww_ = evaluate_baseline_uvw_ha_dec(x, y, z, ha - lon, dec)
        uu, vv, ww = np.append(uu, uu_), np.append(vv, vv_), np.append(ww, ww_)
    layout['uu'], layout['vv'], layout['ww'] = uu, vv, ww
    layout['r2d'] = (uu**2 + vv**2)**0.5    

# Generate uv coordinates for each layout
for i, name in enumerate(layouts):
    layout = layouts[name]
    uvw_coords(layout, obs_length, num_times, lon, dec)

### Plotting of layout and uvw coords

In [13]:
def plot_stations(layouts, name):
    """Plot the station coords from the layout"""
    layout = layouts[name]
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(111)
    for sx, sy in zip(layout['x'], layout['y']):
        c = plt.Circle((sx, sy), station_d/2, fill=True, color='k', 
                       alpha=0.8)
        ax.add_artist(c)     
    c = plt.Circle((0.0, 0.0), r_max, fill=False, color='r', linestyle='-',
                   linewidth=1.5, alpha=0.5)
    ax.add_artist(c)     
    c = plt.Circle((0.0, 0.0), r_min, fill=False, color='r', linestyle='-',
                   linewidth=1.5, alpha=0.5)
    ax.add_artist(c)
    ax.set_xlim(-r_max*1.05, r_max*1.05)
    ax.set_ylim(-r_max*1.05, r_max*1.05)
    # ax.set_title('layout %s' % name)
    ax.set_xlabel('east (m)')
    ax.set_ylabel('north (m)')
    save_fig(fig, 'layout_%s.png' % name, [results_dir, 'layout'])
    # plt.show()
    plt.close(fig)
    
def plot_uvw(layouts, name):
    """Plot baseline uvw coords from the layout"""
    layout = layouts[name]
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(111)
    ax.plot(layout['uu'], layout['vv'], '.', ms=5.0, c='k', alpha=0.2)
    ax.plot(-layout['uu'], -layout['vv'], '.', ms=5.0, c='k', alpha=0.2)
    lim = layout['r2d'].max()
    ax.set_xlim(-lim*1.05, lim*1.05)
    ax.set_ylim(-lim*1.05, lim*1.05)
    # ax.set_title('layout %s' % name)
    ax.set_xlabel('uu (m)')
    ax.set_ylabel('vv (m)')
    save_fig(fig, 'uvw_%s.png' % name, [results_dir, 'uvw'])
    # plt.show()
    plt.close(fig)
    

# Plot layouts
for i, name in enumerate(layouts):
    plot_stations(layouts, name)
    plot_uvw(layouts, name)

### PSFRMS

In [57]:
def pow10floor(x):
    return 10**floor(log10(x))

def pow10round(x):
    return 10**floor(log10(x) + 0.5)

def pow10ceil(x):
    return 10**ceil(log10(x))

def plot_psfrms_r(r, psfrms_r, name):
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(111)
    ax.plot(r, psfrms_r, '.-', ms=10.0, lw=1.0, c='b')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(True)
    ax.set_xlim(pow10floor(r[0]), pow10ceil(r[-1]))
    ax.set_ylim(pow10floor(psfrms_r.min()), 1.05)
    ax.set_xlabel('radius (m)')
    ax.set_ylabel('psfrms')
    save_fig(fig, 'psfrms_%s.png' % name, [results_dir, 'psfrms'])
    # plt.show()
    plt.close(fig)
    
def plot_psfrms_r_compare(layouts):
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(111)
    ymin, xmin, xmax = 1.0e70, 1.0e70, 0.0
    for i, name in enumerate(layouts):
        ax.plot(layouts[name]['psfrms_r_x'], layouts[name]['psfrms_r_y'], 
                '.-', ms=10.0, lw=1.0, label=name)
        ymin = min(ymin, layouts[name]['psfrms_r_y'].min())
        xmin = min(xmin, layouts[name]['psfrms_r_x'][0])
        xmax = max(xmax, layouts[name]['psfrms_r_x'][-1])
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(True)
    ax.set_xlabel('radius (m)')
    ax.set_ylabel('psfrms')
    ax.set_ylim(ymin, 1.1)
    ax.set_xlim(pow10floor(xmin), pow10ceil(xmax))
    ax.set_ylim(pow10floor(ymin), 1.05)
    ax.legend()
    save_fig(fig, 'psfrms_compare.png', [results_dir, 'psfrms'])
    # plt.show()
    plt.close(fig)

def eval_psfrms(layout):
    b_max = layout['r2d'].max() * 2.01
    grid_size = int(ceil(oversampling * (b_max / station_d)))
    if grid_size % 2 == 1:
        grid_size += 1
    cell = b_max / grid_size
    cell_lm = 1.0 / (grid_size * (cell / wavelength))
    lm_max = (grid_size * sin(cell_lm)) / 2.0
    fov = degrees(asin(lm_max)) * 2.0
    imager = oskar.imager.Imager('Single')
    imager.set_grid_kernel('Pillbox', 1, 1)
    imager.set_size(grid_size)
    imager.set_fov(fov)
    uu, vv = layout['uu'], layout['vv']
    uv_grid = np.zeros((grid_size, grid_size), dtype='c8')
    norm = imager.update_plane(uu / wavelength, vv / wavelength,
                               np.zeros_like(uu), 
                               np.ones(uu.shape, dtype='c8'),
                               np.ones_like(uu), uv_grid, 0.0)
    norm += imager.update_plane(-uu / wavelength, -vv / wavelength,
                                np.zeros_like(uu), 
                                np.ones(uu.shape, dtype='c8'),
                                np.ones_like(uu), uv_grid, 0.0)
    if int(norm) != uu.shape[0] * 2 or np.sum(uv_grid.imag) != 0.0:
        raise AssertionError('Gridding error detected %f %i' % 
                             (norm, uu.shape[0] * 2))       
    psfrms = np.sqrt(np.sum(uv_grid.real**2)) / (uu.shape[0] * 2.0)
    layout['psfrms'] = psfrms
    
    # Radial profile
    centre = grid_size // 2
    x_ = np.arange(-centre, centre) * cell
    gx, gy = np.meshgrid(-x_, x_)
    gr = (gx**2 + gy**2)**0.5
    b0 = layout['r2d'].min() if not res_bmin else res_bmin
    b1 = layout['r2d'].max() if not res_bmax else res_bmax
    r_bins = np.logspace(log10(b0), log10(b1), res_n + 1)
    psfrms_r = np.zeros(res_n)
    for i in range(res_n):
        pixels = uv_grid[np.where(gr <= r_bins[i + 1])]
        uv_idx = np.where(layout['r2d'] <= r_bins[i + 1])[0]
        uv_count = uv_idx.shape[0] * 2
        psfrms_r[i] = 1.0 if uv_count == 0 else \
            np.sqrt(np.sum(pixels.real**2)) / uv_count
    layout['psfrms_r_y'] = psfrms_r
    layout['psfrms_r_x'] = r_bins[1:]
    
    
for i, name in enumerate(layouts):
    layout = layouts[name]
    eval_psfrms(layout)
    plot_psfrms_r(layout['psfrms_r_x'], layout['psfrms_r_y'], name)
    

# Plot psfrms radial comparison
plot_psfrms_r_compare(layouts)

### PSF (2d & 1d)

In [114]:
def gen_psf(layout, fov_deg=None, psf_id=0):
    if 'psf' in layout and psf_id in layout['psf']:
        # print('PSF already exists! id=%i' % psf_id)
        return
    b_max = layout['r2d'].max() * 2.01
    grid_size = int(ceil(oversampling * (b_max / station_d)))
    if grid_size % 2 == 1:
        grid_size += 1
    cell = b_max / grid_size
    cell_lm = 1.0 / (grid_size * (cell / wavelength))
    if not fov_deg:
        lm_max = (grid_size * sin(cell_lm)) / 2.0
        fov_deg = degrees(asin(lm_max)) * 2.0
    else:
        lm_max = sin(radians(fov_deg) * 0.5)
        grid_size = int(ceil((lm_max * oversampling * 2) / cell_lm)) * 2
    psf = generate_psf_3(layout['uu'], layout['vv'], layout['ww'], 
                         grid_size, fov_deg)
    if not 'psf' in layout:
        layout['psf'] = dict()
    psf_dict = {
        'image': psf,
        'fov': fov_deg,
        'im_size': grid_size,
        'lm_max': lm_max
    }    
    layout['psf'][psf_id] = psf_dict

    
def plot_psf_2d(psf, name):
    out_name = 'psf_%s_%.3f.png' % (name, psf['fov'])
    out_dir = [results_dir, 'psf_2d']
    path = join(join(*out_dir), out_name)
    if os.path.isfile(join(join(*out_dir), out_name)):
        return
    im_size = psf['im_size']
    centre = im_size / 2
    extent = np.array([centre + 0.5, -centre + 0.5,
                      -centre - 0.5, centre - 0.5])
    lm_inc = (2.0 * psf['lm_max']) / im_size
    extent *= degrees(asin(lm_inc))
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(111)
    im_ = ax.imshow(psf['image'], interpolation='nearest', cmap='inferno', 
                    origin='lower', extent=extent,
                    norm=SymLogNorm(linthresh=0.1, linscale=1.0, vmin=-0.1,
                                   vmax=0.5, clip=False))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.03)
    cbar = ax.figure.colorbar(im_, cax=cax, format='%.1f')
    cbar.ax.tick_params(labelsize='smaller')   
    save_fig(fig, out_name, out_dir, 10, 10)
    # plt.show()
    plt.close(fig)

def get_psf_coords(fov_deg, im_size):
    lm_max = sin(0.5 * radians(fov_deg))
    lm_inc = 2.0 * lm_max / im_size
    l = np.arange(-im_size // 2, im_size // 2, dtype='f8')
    l *= lm_inc
    l, m = np.meshgrid(-l, l)
    r = (l**2 + m**2)**0.5
    return l, m, r
    
def plot_psf_1d(psf, name):
    out_name = 'psf_%s_%.3f.png' % (name, psf['fov'])
    out_dir = [results_dir, 'psf_1d']
    path = join(join(*out_dir), out_name)
    if os.path.isfile(join(join(*out_dir), out_name)):
        return
    l, m, r_lm = get_psf_coords(psf['fov'], psf['im_size'])
    r_lm = r_lm.flatten()
    order = np.argsort(r_lm)
    r_lm = r_lm[order]
    psf_1d = psf['image'].flatten()[order]
    psf_hwhm = (wavelength / (r_max * 2.0)) / 2.0  # FIXME(BM) better expression...
    x = r_lm  # / psf_hwhm
    # x = np.degrees(np.arcsin(r_lm))
    num_bins = 500
    bin_mean, edges, number = \
        stats.binned_statistic(x, psf_1d, statistic='mean', 
                               bins=num_bins)
    bin_x = (edges[1:] + edges[:-1]) / 2
    def bin_max(values):
        return np.abs(values).max()
    bin_max, edges, number = \
        stats.binned_statistic(x, psf_1d, statistic=bin_max, 
                               bins=num_bins)
    psf['bin_x'] = bin_x
    psf['bin_mean'] = bin_mean
    psf['bin_max'] = bin_max
    
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(111)
    ax.plot(bin_x, bin_mean, '-', c='b', lw=1.0, label='mean')
    ax.plot(bin_x, bin_max, '-', c='r', lw=1.0, label='max')
    ax.set_ylim(-0.1, 0.5)
    ax.legend()
    save_fig(fig, out_name, out_dir, 10, 10)
    # plt.show()
    plt.close(fig)

def plot_psf_1d_compare(layouts, psf_id):
    colors = ['r', 'b', 'g']
    fig = plt.figure(figsize=(3, 3))
    ax = fig.add_subplot(111)
    xmax = 0.0
    for i, name in enumerate(layouts):
        psf = layouts[name]['psf'][psf_id]
        ax.plot(psf['bin_x'], psf['bin_max'], '-', c=colors[i%3],
                lw=1.0, label=name)
        ax.plot(psf['bin_x'], psf['bin_mean'], '--', c=colors[i%3], 
                lw=1.0, label=name)
        xmax = max(xmax, psf['bin_x'].max())
    ax.legend()
    ax.set_ylim(-0.1, 0.5)
    ax.set_xlim(0, xmax / 2**0.5)
    out_name = 'psf_compare_%.3f.png' % psf['fov']
    out_dir = [results_dir, 'psf_1d']
    save_fig(fig, out_name, out_dir, 10, 10)
    # plt.show()
    plt.close(fig)
        

# Generate PSF images
for i, name in enumerate(layouts):
    layout = layouts[name]
    gen_psf(layout, psf_id=0)
    # gen_psf(layout, fov_deg=3.0, psf_id=1)
    gen_psf(layout, fov_deg=1.0, psf_id=2)

# Plotting per PSF
for i, name in enumerate(layouts):
    psf = layouts[name]['psf']
    for psf_id in psf:
        plot_psf_2d(psf[psf_id], name)
        plot_psf_1d(psf[psf_id], name)

# Comparison of all layouts for a given id
for psf_id in layouts[layouts.keys()[0]]['psf'].keys():
    plot_psf_1d_compare(layouts, psf_id)
        

### Clear PSF from the layout

In [65]:
for name in layouts:
    layouts[name].pop('psf', None)
    
for name in layouts:
    if 'psf' in layouts[name]:
        print(name, ':', layouts[name]['psf'].keys())
    else:
        print(name, 'has no psf!')

clusters has no psf!
spiral has no psf!
